# Using Machine Learning to identify Enron fraudsters

## Introduction

Enron Corporation was an American energy, commodities, and services company based in Houston, Texas. It was founded in 1985 as the result of a merger between Houston Natural Gas and InterNorth, both relatively small regional companies. Before its bankruptcy on December 2, 2001, Enron employed approximately 20,000 staff and was one of the world's major electricity, natural gas, communications and pulp and paper companies. Fortune named Enron "America's Most Innovative Company" for six consecutive years. At the end of 2001, it was revealed that its reported financial condition was sustained by institutionalized, systematic, and creatively planned accounting fraud, known since as the Enron scandal. Enron has since become a well-known example of willful corporate fraud and corruption.

In this project, I will build a model for identifying potential fraudsters based on financial and e-mail data. For this, the following steps will be performed: 
- data exploration (learning about the data, cleaning and preparing the data)
- feature selection and engineering (selecting the most significant features and creating new ones) 
- reducing the dimensionality of the data using principal component analysis
- selection and tuning a supervised machine learning algorithms 
- validating the algorithm to ensure acceptable performance of the model

The ENRON Email dataset was collected and prepared by the CALO Project (A Cognitive Assistant that Learns and Organizes), see https://www.cs.cmu.edu/~./enron/ for details. The financial data was published in Payments to Insiders report by FindLaw and available at www.findlaw.com

## Understanding the Dataset and Question
### Data exploration

Let's start with importing the data, transforming it into a dataframe, and displaying the header: 

In [2]:
#!/usr/bin/python

import sys
import pickle
import pprint
import numpy as np
import pandas as pd

sys.path.append("../tools/")
import tester
from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data
import tester

### Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)

# Transform dictionary to the Pandas DataFrame  
df = pd.DataFrame.from_dict(data_dict, orient = 'index')
# Names of the features 
features = list(df.keys())
df.head()

,salary,to_messages,deferral_payments,total_payments,exercised_stock_options,bonus,restricted_stock,shared_receipt_with_poi,restricted_stock_deferred,total_stock_value,...,loan_advances,from_messages,other,from_this_person_to_poi,poi,director_fees,deferred_income,long_term_incentive,email_address,from_poi_to_this_person
ALLEN PHILLIP K,201955,2902,2869717,4484442,1729541,4175000,126027,1407,-126027,1729541,...,NaN,2195,152,65,False,NaN,-3081055,304805,phillip.allen@enron.com,47
BADUM JAMES P,NaN,NaN,178980,182466,257817,NaN,NaN,NaN,NaN,257817,...,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN
BANNANTINE JAMES M,477,566,NaN,916197,4046157,NaN,1757552,465,-560222,5243487,...,NaN,29,864523,0,False,NaN,-5104,NaN,james.bannantine@enron.com,39
BAXTER JOHN C,267102,NaN,1295738,5634343,6680544,1200000,3942714,NaN,NaN,10623258,...,NaN,NaN,2660303,NaN,False,NaN,-1386055,1586055,NaN,NaN
BAY FRANKLIN R,239671,NaN,260455,827696,NaN,400000,145796,NaN,-82782,63014,...,NaN,NaN,69,NaN,False,NaN,-201641,NaN,frank.bay@enron.com,NaN


'NaN' values in the dataframe are actually text values, which should be replaces to proper NaN values. Let's make the replacement, look at the features, and analyse how many undefined values do we have:

In [3]:
# Dataframe information
df = df.replace("NaN", np.nan)
df.info()

count = sum(df.count())
count_nan = df.isnull().sum().sum()

print "\nTotal not-null data points:", count
print 'Total NaN: {} ({:.2%})'.format(count_nan, float(count_nan)/float(count+count_nan))

<class 'pandas.core.frame.DataFrame'>
Index: 146 entries, ALLEN PHILLIP K to YEAP SOON
Data columns (total 21 columns):
salary                       95 non-null float64
to_messages                  86 non-null float64
deferral_payments            39 non-null float64
total_payments               125 non-null float64
exercised_stock_options      102 non-null float64
bonus                        82 non-null float64
restricted_stock             110 non-null float64
shared_receipt_with_poi      86 non-null float64
restricted_stock_deferred    18 non-null float64
total_stock_value            126 non-null float64
expenses                     95 non-null float64
loan_advances                4 non-null float64
from_messages                86 non-null float64
other                        93 non-null float64
from_this_person_to_poi      86 non-null float64
poi                          146 non-null bool
director_fees                17 non-null float64
deferred_income              49 non-null float

Let's also look at the allocation across classes: POI (person of interest) vs non-POI. 

In [4]:
# Allocation between classes
poi_types = df.poi.value_counts()
poi_types.index=['non-POI', 'POI']
print "Allocation:\n", poi_types

Allocation:
non-POI    128
POI         18
Name: poi, dtype: int64


As we see: 
- There are 146 entries including 21 features (financial features, email features, and a binary classification "poi"). 
- Among the total of 3066 data points, values are not defined for 1358 (44.29%). We should certainly not be using loan_advances, director_fees, and restricted_stock_deferred for analysis and prediction since these features are not defined for the majority of entries. 
- Among 146 samples, there are 128 non-POI and 18 POI. 

### Outlier Investigation

Now, we will analyse data for the presense of outliers and make a concious decision whether we should keep them in the dataset. We will use the definition from descriptive statistics (points higher than Q3 + 1.5IQR or lower than Q1 - 1.5IQR). Since Enron was a huge corporation with more than twenty thousands of employees, there were probably a lot of people not earning much. Thus, I will only look at the outliers over the upper fence. 

<img src="images/outliers.jpg">

In [5]:
# Upper fence value to detect outliers
upper = df.quantile(.25) + 1.5 * (df.quantile(.75)-df.quantile(.25))

# We should not use features such as email_address and poi for outliers analysis
features_filt = features
features_filt.remove('email_address')
features_filt.remove('poi')

# Finding outliers
upper_o = pd.DataFrame((df[features_filt] > upper[features_filt]).sum(axis = 1), columns = ['outlier_columns_max']).\
    sort_values('outlier_columns_max', ascending = 0)  

print upper_o.head(5)

                    outlier_columns_max
FREVERT MARK A                       11
TOTAL                                11
LAY KENNETH L                        10
LAVORATO JOHN J                      10
WHALLEY LAWRENCE G                    9


Considering the first 5 of the samples with most number of outlier variables:
1. Mark Frevert was named to a position in the chairman's office by Lay Kenneth, [according to Wikipedia](https://en.wikipedia.org/wiki/Enron_scandal). I consider that he represents a valid anomaly and therefore should be kept in the dataset. 
2. The 'TOTAL' should be definitely excluded as it is the total values of all the samples.  
3. Kenneth Lay was a founder, chairman, and CEO, and should be kept in the dataset as a valid anomaly. 
4. John Lavorato was a chief executive of Enron Americas who received $5 mUSD as a retention bonus according to a [CNN article](http://edition.cnn.com/2002/LAW/02/09/enron.bonuses/). I do not think that he should be excluded. 
5. Lawrence 'Greg' Whalley was an Enron president and chief operating officer for a short period of time, and represents a valid anomaly. 

In [6]:
# Exclude an outlier
df = df.drop(['TOTAL'])

Additionally, let's take a quick skim through on the indexes of our dataframe to detect anything unusual or incorrect. 

In [7]:
# Get indexes
df.index.values

array(['ALLEN PHILLIP K', 'BADUM JAMES P', 'BANNANTINE JAMES M',
       'BAXTER JOHN C', 'BAY FRANKLIN R', 'BAZELIDES PHILIP J',
       'BECK SALLY W', 'BELDEN TIMOTHY N', 'BELFER ROBERT',
       'BERBERIAN DAVID', 'BERGSIEKER RICHARD P', 'BHATNAGAR SANJAY',
       'BIBI PHILIPPE A', 'BLACHMAN JEREMY M', 'BLAKE JR. NORMAN P',
       'BOWEN JR RAYMOND M', 'BROWN MICHAEL', 'BUCHANAN HAROLD G',
       'BUTTS ROBERT H', 'BUY RICHARD B', 'CALGER CHRISTOPHER F',
       'CARTER REBECCA C', 'CAUSEY RICHARD A', 'CHAN RONNIE',
       'CHRISTODOULOU DIOMEDES', 'CLINE KENNETH W', 'COLWELL WESLEY',
       'CORDES WILLIAM R', 'COX DAVID', 'CUMBERLAND MICHAEL S',
       'DEFFNER JOSEPH M', 'DELAINEY DAVID W', 'DERRICK JR. JAMES V',
       'DETMERING TIMOTHY J', 'DIETRICH JANET R', 'DIMICHELE RICHARD G',
       'DODSON KEITH', 'DONAHUE JR JEFFREY M', 'DUNCAN JOHN H',
       'DURAN WILLIAM D', 'ECHOLS JOHN B', 'ELLIOTT STEVEN',
       'FALLON JAMES B', 'FASTOW ANDREW S', 'FITZGERALD JAY L',
       'FOW

The index 'THE TRAVEL AGENCY IN THE PARK' is definitely not a person - let's also exclude it. 

In [8]:
# Exclude an outlier
df = df.drop(['THE TRAVEL AGENCY IN THE PARK'])

## Optimize Feature Selection/Engineering
### Create new features

Firstly, let's reiterate on available features and completeness of information for POI and non-POI. As analysed above, we will not be using loan_advances, director_fees, and restricted_stock_deferred features as they are too incomplete. We will drop them from the dataframe. Also, email_address identifies the person and is unique - we will discard this feature too. 

In [9]:
df = df.drop(['loan_advances','director_fees','restricted_stock_deferred', 'email_address'], 1)

Now, looking at the other features for POI and non-POI: 

In [10]:
df[(df.poi == True)].info()

<class 'pandas.core.frame.DataFrame'>
Index: 18 entries, BELDEN TIMOTHY N to YEAGER F SCOTT
Data columns (total 17 columns):
salary                     17 non-null float64
to_messages                14 non-null float64
deferral_payments          5 non-null float64
total_payments             18 non-null float64
exercised_stock_options    12 non-null float64
bonus                      16 non-null float64
restricted_stock           17 non-null float64
shared_receipt_with_poi    14 non-null float64
total_stock_value          18 non-null float64
expenses                   18 non-null float64
from_messages              14 non-null float64
other                      18 non-null float64
from_this_person_to_poi    14 non-null float64
poi                        18 non-null bool
deferred_income            11 non-null float64
long_term_incentive        12 non-null float64
from_poi_to_this_person    14 non-null float64
dtypes: bool(1), float64(16)
memory usage: 2.4+ KB


In [11]:
df[(df.poi == False)].info()

<class 'pandas.core.frame.DataFrame'>
Index: 126 entries, ALLEN PHILLIP K to YEAP SOON
Data columns (total 17 columns):
salary                     77 non-null float64
to_messages                72 non-null float64
deferral_payments          33 non-null float64
total_payments             105 non-null float64
exercised_stock_options    89 non-null float64
bonus                      65 non-null float64
restricted_stock           92 non-null float64
shared_receipt_with_poi    72 non-null float64
total_stock_value          107 non-null float64
expenses                   76 non-null float64
from_messages              72 non-null float64
other                      73 non-null float64
from_this_person_to_poi    72 non-null float64
poi                        126 non-null bool
deferred_income            37 non-null float64
long_term_incentive        53 non-null float64
from_poi_to_this_person    72 non-null float64
dtypes: bool(1), float64(16)
memory usage: 16.9+ KB


This looks fine. Now, considering the relationships between features and relativity of their values, we will create a set of new ones. For instance, salary is a part of total_payments so the fraction of them intuitively feels like a better indicator for comparison betweeen employees. We will add the following features for further selection of the best ones (on the step 'intelligently select features'): 
- f_bonus (bonus/total_payments)
- f_salary (salary/total_payments)
- f_long_term_incentive (long_term_incentive/total_payments)
- f_exercised_stock_options (exercised_stock_options/total_stock_value)
- f_restricted_stock (restricted_stock/total_stock_value)

In [12]:
df['f_bonus'] = df['bonus']/df['total_payments']
df['f_salary'] = df['salary']/df['total_payments']
df['f_long_term_incentive'] = df['long_term_incentive']/df['total_payments']
df['f_exercised_stock_options'] = df['exercised_stock_options']/df['total_stock_value']
df['f_restricted_stock'] = df['restricted_stock']/df['total_stock_value']

Similarly, we can create new indicators for e-mails: 

In [13]:
df['f_from_poi'] = df['from_poi_to_this_person']/df['to_messages']
df['f_to_poi'] = df['from_this_person_to_poi']/df['from_messages']
df['f_shared_receipt_with_poi'] = df['shared_receipt_with_poi']/df['to_messages']

### Properly scale features

I have decided to apply scaling for the dataset. While scaling is not necessary for all of the machine learning algorithms in general, it would allow for greater flexibility moving further and applying different algorithms. We can use the following table for future reference (source - [stackexchange](https://stats.stackexchange.com/questions/244507/what-algorithms-need-feature-scaling-beside-from-svm)):

| Algorithm           | Features need scaling   |
|---------------------|-------------------------|
| KNN                 | Yes                     |
| Linear regression   | No (unless regularized) |
| Logistic regression | No (unless regularized) |
| Naive Bayes         | No                      |
| Decision trees      | No                      |
| Random forests      | No                      |
| AdaBoost            | No                      |
| Neural networks     | Yes                     |

Min-Max scaling will be used to scale the features. Since it does not work with NaN values, we will have to replace them with some values. I have not found any information on the Internet of whether missing values represent zeroes - thus, I will fill them with mean values and then scale: 

In [14]:
from sklearn.preprocessing import MinMaxScaler
from pandas import DataFrame

df = df.fillna(df.mean())

# Preserving the dataframe structure
i = df.index
c = df.columns

# Scaling
scaler = MinMaxScaler()
scaler.fit(df)

df = DataFrame(scaler.transform(df), index=i, columns=c)
df.head()

,salary,to_messages,deferral_payments,total_payments,exercised_stock_options,bonus,restricted_stock,shared_receipt_with_poi,total_stock_value,expenses,...,long_term_incentive,from_poi_to_this_person,f_bonus,f_salary,f_long_term_incentive,f_exercised_stock_options,f_restricted_stock,f_from_poi,f_to_poi,f_shared_receipt_with_poi
ALLEN PHILLIP K,0.181384,0.188510,0.455199,0.043302,0.050262,0.517654,0.157232,0.254575,0.036083,0.060014,...,0.046409,0.089015,0.168356,0.046984,0.006235,1.000000,0.016593,0.074518,0.029613,0.474640
BADUM JAMES P,0.255325,0.133638,0.043109,0.001761,0.007411,0.142720,0.216047,0.212804,0.006142,0.014601,...,0.133420,0.122908,0.105040,0.243339,0.069352,1.000000,0.153338,0.174670,0.184055,0.601038
BANNANTINE JAMES M,0.000000,0.033726,0.144591,0.008846,0.117713,0.142720,0.251180,0.083892,0.107571,0.245623,...,0.133420,0.073864,0.105040,0.000000,0.069352,0.787486,0.092007,0.317034,0.000000,0.817260
BAXTER JOHN C,0.240034,0.133638,0.214142,0.054405,0.194417,0.142497,0.377009,0.212804,0.217018,0.048343,...,0.298812,0.122908,0.028349,0.049487,0.044584,0.654594,0.102343,0.174670,0.184055,0.601038
BAY FRANKLIN R,0.215339,0.133638,0.055587,0.007991,0.086076,0.041614,0.158370,0.212804,0.002179,0.564241,...,0.133420,0.122908,0.081053,0.305082,0.069352,0.737972,0.660813,0.174670,0.184055,0.601038


### Intelligently select features 

Before moving forward, we need to transform data so it can be used with preprocessing functions which I used in the course of [Udacity Data Analyst Nanodegree](https://www.udacity.com/course/data-analyst-nanodegree--nd002). 

In [15]:
# Convert to a dictionary and use preprocessing functions. 
my_list = df.to_dict(orient='records')
my_dataset = {}

for index, values in zip(i, my_list):
    my_dataset[index] = values

# We will need a full list of features to select the best ones  
features_list_full = my_dataset.itervalues().next().keys()

# POI at the first position for further use with preprocessing functions 
features_list_full.remove('poi')
features_used = list(features_list_full)
features_list_full.insert(0, 'poi')

# Use preprocessing functions 
data = featureFormat(my_dataset, features_list_full)
labels, features = targetFeatureSplit(data)

Now, let's create and run a function which iterates over different samples and selects 10 most significant features using DecisionTreeClassifier and SelectKBest.

Firstly, let's think of all the available features. We have introduced some derived features above. Apparently, we will need to remove either original or derived features depending on their significance. I have written a helper function which can be used to do this. 

In [92]:
# Function to remove derived or original values depending on their strength
feature_singlets = {'f_bonus':'bonus', 'f_salary':'salary', 
                    'f_long_term_incentive':'long_term_incentive', 
                    'f_exercised_stock_options':'exercised_stock_options',
                    'f_restricted_stock':'restricted_stock'
                    }

feature_pairs = {'f_from_poi':['from_poi_to_this_person', 'to_messages'], 
                 'f_to_poi':['from_this_person_to_poi', 'from_messages'], 
                 'f_shared_receipt_with_poi':['shared_receipt_with_poi', 'to_messages']
                }

def rm_excessive_features(features_source, feature_pairs, feature_singlets):
    rm_values = set()
    
    for key, value in feature_pairs.iteritems(): 
        # Calculating original and derived weight 
        # Comparing by weighted average
        w_org = (features_source[value[0]] + features_source[value[1]])/2
        w_drv = features_source[key]
        if w_drv >= w_org: 
            #print 'Removing feature:', value[0], value[1]
            rm_values.add(value[0])
            rm_values.add(value[1])                                  
        else: 
            #print 'Removing feature:', key
            rm_values.add(key)  
    
    for key, value in feature_singlets.iteritems(): 
        w_org = features_source[value] 
        w_drv = features_source[key]
        if w_drv >= w_org: 
            #print 'Removing feature:', value
            rm_values.add(value)                            
        else: 
            #print 'Removing feature:', key
            rm_values.add(key)     
            
    # Removing less significant features
    for rm_val in rm_values: 
        features_source.pop(rm_val, None)    
    
    return features_source

Now, I will write a finction to produce shuffled folds, detect the most significant features, and removing excessive features. 

In [135]:
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn import tree
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
# To sort the result
import operator

# Create a function to return combinations of the most significant features 
def select_features(features_used, features, labels): 
    # Create a dictionary to populate from feature_used array, and a results list

    best_f_dict = dict.fromkeys(features_used, 0)
    results = []

    # Create a Stratified ShuffleSplit cross-validaton with 10 splits (folds)
    # 10 is a default value, which should be enough for a dataset containing 144 values
    cv = StratifiedShuffleSplit(labels, 10)
    #print "\nFeature engineering:"
    
    # Try classifiers on folds and select the best features
    for train_idx, test_idx in cv:
        features_train = []
        features_test  = []
        labels_train   = []
        labels_test    = []

        for ii in train_idx:
            features_train.append( features[ii] )
            labels_train.append( labels[ii] )
        for jj in test_idx:
            features_test.append( features[jj] )
            labels_test.append( labels[jj] )

        # Create a Decision Tree classifier 
        clf = tree.DecisionTreeClassifier()    
        # Fit the classifier using training set, and test on test set
        clf.fit(features_train, labels_train)
        # Detect most important features from the tree
        importances = clf.feature_importances_
        indices = np.argsort(importances)[::-1]   
        # Get 10 most important features of the Decision tree 
        for i in range(10):
            best_f_val = features_used[indices[i]]
            best_f_dict[best_f_val] = best_f_dict[best_f_val] + 1

        # Detect most important features using SelectKBest
        selector = SelectKBest(chi2, k=10)
        selector.fit(features_train, labels_train)
        features_train_transformed = selector.transform(features_train) 
        support = np.asarray(selector.get_support())

        # Get an array with the most important features 
        features_arr = np.asarray(features_used)
        columns_support = features_arr[support]
        for val in columns_support: 
            best_f_dict[val] = best_f_dict[val] + 1

    # Remove less significant features 
    best_f_dict = rm_excessive_features(best_f_dict, feature_pairs, feature_singlets)      
            
    # Get the list of sorted features sorted by their importance 
    best_f_list = sorted(best_f_dict.iteritems(), key=operator.itemgetter(1), reverse=True)
    
    #print "\nThe most important features by scores:"
    #pprint.pprint(best_f_list)

    # Save the list of features for future reference, sorted by their importance
    features_filtered = []
    for val in best_f_list: 
        features_filtered.append(val[0])
        
    results = features_filtered 
    return results 

# Generate a list of sets of the most significant features to evaluate performance on different combinations 
features_filtered_set = []
for i in range (1, 6):
    best_set_found = select_features(features_used, features, labels)
    print "\nFeature combination", i, best_set_found
    features_filtered_set.append(best_set_found)


Feature combination 1 ['f_bonus', 'deferral_payments', 'f_from_poi', 'f_salary', 'deferred_income', 'other', 'f_shared_receipt_with_poi', 'from_messages', 'expenses', 'from_this_person_to_poi', 'long_term_incentive', 'total_payments', 'f_exercised_stock_options', 'f_restricted_stock', 'total_stock_value']

Feature combination 2 ['f_salary', 'deferral_payments', 'expenses', 'deferred_income', 'f_bonus', 'other', 'f_shared_receipt_with_poi', 'from_poi_to_this_person', 'from_messages', 'from_this_person_to_poi', 'long_term_incentive', 'f_exercised_stock_options', 'total_payments', 'f_restricted_stock', 'total_stock_value']

Feature combination 3 ['to_messages', 'expenses', 'f_salary', 'deferral_payments', 'f_bonus', 'shared_receipt_with_poi', 'other', 'deferred_income', 'from_poi_to_this_person', 'long_term_incentive', 'from_messages', 'restricted_stock', 'from_this_person_to_poi', 'exercised_stock_options', 'total_payments', 'total_stock_value']

Feature combination 4 ['deferral_payment

We will use the selected features further. 

## Pick and Tune an Algorithm
### Pick an algorithm

I dediced to pick three algorithms: Decision Tree, Gaussian Naive Bayes, and K-nearest neighbours. First, I will build a few auxiliary functions and then implement Decision Tree classifier: 

In [129]:
from sklearn.pipeline import Pipeline

# Function which return shuffled testing and training sets 
def shuffle_split(labels, features): 
    features_train = []
    features_test  = []
    labels_train   = []
    labels_test    = []   
    # 10 is a default value, which should be enough for a dataset containing 144 values
    cv = StratifiedShuffleSplit(labels, 10)
    # Create leatures and labels shuffled dataset
    for train_idx, test_idx in cv:    
        for ii in train_idx:
            features_train.append( features[ii] )
            labels_train.append( labels[ii] )
        for jj in test_idx:
            features_test.append( features[jj] )
            labels_test.append( labels[jj] )
    return features_train, features_test, labels_train, labels_test

# Function to produce shuffled sets for each of identified features combination and to fit / evaluate a classifier on each
def prepare_evaluate(my_dataset, features_filtered_set, clf):
    i = 1
    for features_filtered in features_filtered_set:       
        # Prepare features list for tester and for data split 
        features_tester = list(features_filtered)
        features_tester.insert(0, 'poi')
        data = featureFormat(my_dataset, features_tester)
        labels, features = targetFeatureSplit(data)
        # Get the shuffled sets
        features_train, features_test, labels_train, labels_test = shuffle_split(labels, features)
        print "Evaluating classifier on the feature set", i
        # Fit the classifier
        clf.fit(features_train, labels_train)
        # Call tester 
        tester.test_classifier(clf, my_dataset, features_tester)
        i += 1

In [136]:
# Create and use a Decision Tree classifier 
clf_tree = tree.DecisionTreeClassifier()
prepare_evaluate(my_dataset, features_filtered_set, clf_tree)

Evaluating classifier on the feature set 1
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
	Accuracy: 0.82160	Precision: 0.32631	Recall: 0.31750	F1: 0.32184	F2: 0.31922
	Total predictions: 15000	True positives:  635	False positives: 1311	False negatives: 1365	True negatives: 11689

Evaluating classifier on the feature set 2
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
	Accuracy: 0.81460	Precision: 0.30582	Recall: 0.30750	F1: 0.30666	F2: 0.30716
	Total predictions: 

GaussianNB:

In [137]:
from sklearn.naive_bayes import GaussianNB

# Create and use a GaussianNB classifier 
clf_gnb = GaussianNB()
prepare_evaluate(my_dataset, features_filtered_set, clf_gnb)

Evaluating classifier on the feature set 1
GaussianNB(priors=None)
	Accuracy: 0.83127	Precision: 0.32359	Recall: 0.24350	F1: 0.27789	F2: 0.25618
	Total predictions: 15000	True positives:  487	False positives: 1018	False negatives: 1513	True negatives: 11982

Evaluating classifier on the feature set 2
GaussianNB(priors=None)
	Accuracy: 0.83193	Precision: 0.32575	Recall: 0.24350	F1: 0.27868	F2: 0.25645
	Total predictions: 15000	True positives:  487	False positives: 1008	False negatives: 1513	True negatives: 11992

Evaluating classifier on the feature set 3
GaussianNB(priors=None)
	Accuracy: 0.82800	Precision: 0.31761	Recall: 0.25250	F1: 0.28134	F2: 0.26330
	Total predictions: 15000	True positives:  505	False positives: 1085	False negatives: 1495	True negatives: 11915

Evaluating classifier on the feature set 4
GaussianNB(priors=None)
	Accuracy: 0.82873	Precision: 0.31562	Recall: 0.24350	F1: 0.27491	F2: 0.25516
	Total predictions: 15000	True positives:  487	False positives: 1056	False neg

GaussianNB with feature selection and PCA

In [138]:
# Creating a pipeline with PCA and the GaussianNB
pca = PCA(n_components=3)
clf_gnb_n = GaussianNB()
pipe = Pipeline([('pca', pca), ('model', clf_gnb_n)])

prepare_evaluate(my_dataset, features_filtered_set, pipe)

Evaluating classifier on the feature set 1
Pipeline(steps=[('pca', PCA(copy=True, iterated_power='auto', n_components=3, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('model', GaussianNB(priors=None))])
	Accuracy: 0.85093	Precision: 0.22685	Recall: 0.04900	F1: 0.08059	F2: 0.05811
	Total predictions: 15000	True positives:   98	False positives:  334	False negatives: 1902	True negatives: 12666

Evaluating classifier on the feature set 2
Pipeline(steps=[('pca', PCA(copy=True, iterated_power='auto', n_components=3, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('model', GaussianNB(priors=None))])
	Accuracy: 0.86160	Precision: 0.39730	Recall: 0.07350	F1: 0.12405	F2: 0.08781
	Total predictions: 15000	True positives:  147	False positives:  223	False negatives: 1853	True negatives: 12777

Evaluating classifier on the feature set 3
Pipeline(steps=[('pca', PCA(copy=True, iterated_power='auto', n_components=3, random_state=None,
  svd_solver='auto', tol=0.0, 

K-nearest neighbours:

In [140]:
from sklearn.neighbors import KNeighborsClassifier

# Create and use a kNN classifier 
clf_knn = KNeighborsClassifier()
prepare_evaluate(my_dataset, features_filtered_set, clf_knn)

Evaluating classifier on the feature set 1
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')
	Accuracy: 0.86687	Precision: 1.00000	Recall: 0.00150	F1: 0.00300	F2: 0.00187
	Total predictions: 15000	True positives:    3	False positives:    0	False negatives: 1997	True negatives: 13000

Evaluating classifier on the feature set 2
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')
	Accuracy: 0.87267	Precision: 0.98913	Recall: 0.04550	F1: 0.08700	F2: 0.05623
	Total predictions: 15000	True positives:   91	False positives:    1	False negatives: 1909	True negatives: 12999

Evaluating classifier on the feature set 3
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='unifo

Summary of the results for the first feature set is: 

| Algorithm                                   | Precision | Recall  | F1      |
|---------------------------------------------|-----------|---------|---------|
| Decision trees                              | 0.32631   | 0.31750 | 0.32184 |
| GaussianNB                                  | 0.32359   | 0.24350 | 0.27789 |
| GaussianNB (with PCA and feature selection) | 0.22685   | 0.04900 | 0.08059 |
| K Nearest Neighbours                        | 1.00000   | 0.00150 | 0.00300 |

For all the feature sets, comparing F1 score: 

| Algorithm                                   | F1: set 1 | F1: set 2 | F1: set 3 | F1: set 4 | F1: set 5 |
|---------------------------------------------|-----------|-----------|-----------|-----------|-----------|
| Decision trees                              | 0.32184   | 0.30666   | 0.28919   | 0.29559   | 0.22905   |
| GaussianNB                                  | 0.27789   | 0.27868   | 0.28134   | 0.27491   | 0.28120   |
| GaussianNB (with PCA and feature selection) | 0.08059   | 0.12405   | 0.33251   | 0.15372   | 0.13850   |
| K Nearest Neighbours                        | 0.00300   | 0.08700   | 0.08724   | 0.08630   | 0.11346   |

Decision trees looks to perform the best across different feature sets, giving the highest F1 score in average. Gaussian Naive Bayes with PCA and feature selection, as well as kNN, give quite poor results for F1. Looking at the precision value for kNN on the first feature set, seems that it is prone to overfitting in our case. Let's proceed with the decision tree and fine-tune the algorithm. I will proceed with the first feature set as it gives the best results for this algorithm. 

In [142]:
features_filtered = features_filtered_set[0]

### Parameter tuning

A general complication in Machine Learning is that learning algorithms require us to set parameters before using the models, and suboptimal selection of parameters can significantly influence the performance (as we saw above with the GaussianNB classifier). Our goal should be to optimise the parameters that impact the model in order to enable the algorithm to perform the best ("best" can be defined with metrics such as recall, precision, and F1 score). 

Sklearn [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) is a way of systematically working through multiple combinations of parameter tunes, cross-validating as it goes to determine which tune gives the best performance. We can use it to tune the Decision tree classifier. I decided to select the following parameters for tuning:
- criterion (the function to measure the quality of a split),
- splitter (the strategy used to choose the split at each node), 
- min_samples_split (the minimum number of samples required to split an internal node), 
- min_samples_leaf (the minimum number of samples required to be at a leaf node). 

In [159]:
from sklearn import svm, grid_search

# Parameters for fine tuning  
parameters = {'splitter':('best','random'), 
    'min_samples_split':(2, 5, 10, 15), 
    'min_samples_leaf':(1, 3, 6, 8, 10)}

# Update the dataset
features_tester = list(features_filtered)
features_tester.insert(0, 'poi')
data = featureFormat(my_dataset, features_tester)
labels, features = targetFeatureSplit(data)
features_train, features_test, labels_train, labels_test = shuffle_split(labels, features)

# Create and use a classifier for fitting
clf_tree = tree.DecisionTreeClassifier()
clf_tree = grid_search.GridSearchCV(clf_tree, parameters) 

clf_tree.fit(features_train, labels_train)

print "Best parameters:", clf_tree.best_params_

Best parameters: {'min_samples_split': 2, 'splitter': 'best', 'min_samples_leaf': 1}


In [160]:
# Using the parameters
clf_tree = tree.DecisionTreeClassifier(min_samples_split = 2,
                             splitter = 'best',
                             min_samples_leaf = 1)

clf_tree.fit(features_train, labels_train)

# Call tester 
tester.dump_classifier_and_data(clf_tree, my_dataset, features_tester)
tester.main() 

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
	Accuracy: 0.81927	Precision: 0.31834	Recall: 0.31150	F1: 0.31489	F2: 0.31285
	Total predictions: 15000	True positives:  623	False positives: 1334	False negatives: 1377	True negatives: 11666



## Validate and evaluate 
### Validation and its importance

One of the most important notions of data science are related to overfitting and generalization. If we allow ourselves enough flexibility in searching for patterns in a particular dataset, we will find patterns. Generally, we are interested in patterns
that generalize—that predict well for instances that we have not yet observed. If the patterns do not generalise, then we have created a model which is overfit. 

To avoid overfitting, validation is used to build and test a model. In this process, a trained model is evaluated with a testing subset, which is 'hold out' of the original dataset not used for training. The main purpose of using the testing data set is to test the generalisation ability of a trained model. 

However, creating training and testing sets by splitting without proper data shuffle could lead to uneven mix of data across classes, especially if data is skewed or grouped sequentally in the original dataset. Cross-validation is a proven approach to deal with this issue. Unlike splitting the data into one training and one holdout set, cross-validation computes its estimates over all the data by performing multiple splits and systematically swapping out samples for testing. Cross-validation begins by splitting a labeled dataset into k partitions called folds, and then then iterates training and testing k times when a different fold is chosen as the test data in each iteration of the cross-validation. 

In sklearn, [StratifiedShuffleSplit](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html) can be used to implement cross-validation. This has been included in the testing function (tester.py), which shuffles the data and splits it into 1000 folds. We have already used StratifiedShuffleSplit for sets preparation in order to select the most significant features, train different classifiers, and fine-tune the best performing one. 

### Evaluation Metrics and Algorithm Performance

There are several different ways to measure the performance of a Machine Learning algorithm, and the most commonly used are precision, recall, and the F1-score. 

<img src="images/pr_rec_f.png">
- Precision: the number of times the algorithm positively identifies a data point (known as a true positive) divided by the total number of positives (regardless of whether they were correct). High precision value means that POIs identified by the algorithm tended to be correct, while a low value means there were more false alarms (non-POIs flagged as POIs).
- Recall: number of events you can correctly recall divided by the the number of all correct events. In other words, this means that the number of correctly identified POIs (true positives) is divided by the number of POIs in the dataset (true positives + false negatives). A false negative would be represented by labeling a POI as a non-POI. 
- F1 score: a weighted average of precision and recall. It’s calculated by multiplying the product of the recall and precision by two, and then dividing by the sum of the precision and recall. In our case, both precision and recall values of 0.3 had to be achieved using the final algorithm. 

To conclude, the following features were selected for the model: 
- f_bonus
- deferral_payments
- f_from_poi
- f_salary
- deferred_income
- other
- f_shared_receipt_with_poi
- from_messages
- expenses
- from_this_person_to_poi
- long_term_incentive
- total_payments
- f_exercised_stock_options
- f_restricted_stock
- total_stock_value 

Decision Tree Classifier algorithm was used with the parameters min_samples_split : 2, splitter : 'best', min_samples_leaf : 1 (which is default).

Results of evaluation using cross-validation on 1000 folds:
- Precision: 0.31834
- Recall: 0.31150
- F1 score: 0.31489